# Graph Generation

In [1]:
import utils as utl
import pickle as pkl
import pandas as pd
import numpy as np

path2root = '.'

## Load data

In [2]:
recipes_similarities = pkl.load(open(f'{path2root}/data/tf_idf_similarities.pkl', 'rb'))
comb_recipe_similarities = pkl.load(open(f'{path2root}/data/comb_recipe_similarities.pkl', 'rb'))
embeddings_dict = pkl.load(open(f'{path2root}/data/initial_features/initial_features_dict.pkl', 'rb'))
recipes_embeddings, combinations_embeddings = embeddings_dict['recipes'], embeddings_dict['combinations']

## Generate index

In [17]:
embeddings = pd.concat([recipes_embeddings, combinations_embeddings])

In [18]:
index = embeddings.reset_index()['index']
index_to_serial = dict(index)
serial_to_index = dict(pd.DataFrame(index).reset_index().set_index('index')['level_0'])

## Comb-Recipe edges

In [5]:
comb_recipe_similarities.index = comb_recipe_similarities.index.map(serial_to_index.get)
comb_recipe_similarities.columns = comb_recipe_similarities.columns.map(serial_to_index.get)
comb_recipe_similarities

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
10000,0.134140,0.169741,0.063430,0.000000,0.064222,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.072595,0.000000,0.199347,0.382372,0.000000,0.000000,0.353445,0.296873,0.000000,0.000000
10001,0.184628,0.206017,0.096273,0.027116,0.095184,0.027088,0.006531,0.013802,0.018500,0.018393,...,0.109061,0.006277,0.220693,0.404388,0.022032,0.026603,0.372927,0.318095,0.040554,0.016720
10002,0.233712,0.240924,0.128320,0.053831,0.125373,0.053775,0.012965,0.027401,0.036728,0.036515,...,0.144631,0.012461,0.240746,0.424201,0.043738,0.052813,0.390387,0.337547,0.080509,0.033194
10003,0.280740,0.274014,0.159139,0.079775,0.154385,0.079692,0.019214,0.040606,0.054429,0.054114,...,0.178829,0.018466,0.259269,0.441621,0.064818,0.078266,0.405658,0.355021,0.119310,0.049192
10004,0.325179,0.304939,0.188374,0.104629,0.181885,0.104520,0.025200,0.053257,0.071386,0.070973,...,0.211258,0.024220,0.276104,0.456562,0.085011,0.102650,0.418673,0.370406,0.156481,0.064517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15440,0.663900,0.645747,0.722567,0.178792,0.609933,0.460318,0.504392,0.080338,0.392391,0.633871,...,0.312108,0.367003,0.511279,0.148329,0.608488,0.208562,0.386226,0.411571,0.286629,0.099426
15441,0.691488,0.661675,0.733483,0.197723,0.621944,0.472465,0.497875,0.090230,0.398815,0.634405,...,0.336001,0.363567,0.517711,0.164440,0.612748,0.226340,0.394393,0.420499,0.314481,0.111360
15442,0.715687,0.674880,0.741716,0.215137,0.631543,0.482631,0.490229,0.099368,0.403756,0.633061,...,0.357727,0.359241,0.522312,0.179270,0.615008,0.242573,0.401003,0.427756,0.340031,0.122375
15443,0.736713,0.685609,0.747565,0.231052,0.638973,0.490986,0.481710,0.107755,0.407374,0.630132,...,0.377349,0.354206,0.525299,0.192834,0.615536,0.257300,0.406208,0.433503,0.363318,0.132479


In [6]:
print(utl.edge_amount)
if utl.edge_amount == 'threshold':
    threshold = utl.threshold
    melted = pd.melt(comb_recipe_similarities, var_name='recipe', ignore_index=False).reset_index(names='combination')
    melted = melted[melted['value'] >= threshold]
    edge_indexA = melted[['combination', 'recipe']].values.T
elif utl.edge_amount == 'number':
    degree = utl.comb_degree
    edge_indexA = comb_recipe_similarities.apply(lambda s: s.sort_values(ascending=False).index[:5].to_list(),
                                                 axis=1).explode().reset_index().values.T
else:
    print("Error")
print(edge_indexA.shape)

threshold
(2, 17568)


## Recipe-Recipe edges

In [7]:
recipes_similarities.index = recipes_similarities.index.map(serial_to_index.get)
recipes_similarities.columns = recipes_similarities.columns.map(serial_to_index.get)
recipes_similarities

,3031,1821,6062,3880,8433,2356,9125,902,7686,8322,...,2898,7485,1083,6010,6955,2155,252,7267,5513,2483
3031,1.000000,0.035413,0.026217,0.014202,0.051022,0.011517,0.005737,0.014152,0.031075,0.061042,...,0.007868,0.019565,0.029663,0.019063,0.057461,0.035567,0.005617,0.012290,0.005863,0.021575
1821,0.035413,1.000000,0.007001,0.031480,0.076984,0.023355,0.039113,0.038409,0.028761,0.017359,...,0.030560,0.034583,0.025012,0.018956,0.018739,0.061224,0.023737,0.021546,0.014598,0.029312
6062,0.026217,0.007001,1.000000,0.081927,0.017332,0.207157,0.008605,0.043884,0.037566,0.096308,...,0.032897,0.057700,0.032771,0.031193,0.045485,0.070115,0.039574,0.027027,0.036325,0.074607
3880,0.014202,0.031480,0.081927,1.000000,0.059339,0.184654,0.068163,0.102089,0.113964,0.027635,...,0.038491,0.061879,0.065827,0.033855,0.049380,0.319063,0.073876,0.032556,0.051875,0.092907
8433,0.051022,0.076984,0.017332,0.059339,1.000000,0.098044,0.033297,0.056125,0.075210,0.036086,...,0.024760,0.104892,0.055313,0.056337,0.057982,0.064855,0.261114,0.090482,0.064953,0.057464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2155,0.035567,0.061224,0.070115,0.319063,0.064855,0.271727,0.050959,0.061205,0.095004,0.068147,...,0.074420,0.068578,0.024816,0.074241,0.053551,1.000000,0.064713,0.049665,0.107490,0.122026
252,0.005617,0.023737,0.039574,0.073876,0.261114,0.100486,0.178428,0.010146,0.232419,0.066510,...,0.018859,0.169763,0.020962,0.095908,0.097255,0.064713,1.000000,0.061114,0.150152,0.017413
7267,0.012290,0.021546,0.027027,0.032556,0.090482,0.060427,0.062096,0.018168,0.015272,0.060530,...,0.018287,0.032521,0.022854,0.106004,0.077359,0.049665,0.061114,1.000000,0.088757,0.016862
5513,0.005863,0.014598,0.036325,0.051875,0.064953,0.142748,0.157363,0.026358,0.078139,0.030425,...,0.036260,0.044222,0.017714,0.079828,0.058991,0.107490,0.150152,0.088757,1.000000,0.012730


In [8]:
print(utl.edge_amount)
if utl.edge_amount == 'threshold':
    threshold = utl.threshold
    melted = pd.melt(recipes_similarities, var_name='recipe', ignore_index=False).reset_index(names='combination')
    melted = melted[melted['value'] >= threshold]
    edge_indexB = melted[['combination', 'recipe']].values.T
elif utl.edge_amount == 'number':
    num_edges = utl.num_edges
    keep = np.triu(np.ones(recipes_similarities.shape), k=1).astype('bool').reshape(recipes_similarities.size)
    melted = recipes_similarities.stack()[keep]
    edge_indexB = melted.nlargest(n=num_edges, keep='first').reset_index()[['level_0', 'level_1']].values.T
else:
    print("Error")
print(edge_indexB.shape)

threshold
(2, 10078)


## Merge

In [14]:
edge_index = np.concatenate([edge_indexA, edge_indexB], axis=1)

In [20]:
embeddings.index = embeddings.index.map(serial_to_index.get)
embeddings = embeddings.values
embeddings

In [22]:
num_nodes = embeddings.shape[0]
num_nodes

15445

In [30]:
G = {'edge_index': edge_index,
     'num_nodes': num_nodes,
     'embeddings': embeddings,
     'recipes_similarities': recipes_similarities,
     'comb_recipe_similarities': comb_recipe_similarities,
     'serial_to_index': serial_to_index,
     'index_to_serial': index_to_serial}

In [31]:
pkl.dump(G, open(f'{path2root}/data/G_data.pkl', 'wb'))